#Install

In [ ]:
import pandas as pd
from pathlib import Path
import numpy as np
import pandas as pd
import warnings
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
warnings.filterwarnings("ignore")

In [ ]:
from sklearn.metrics import f1_score, accuracy_score

In [ ]:
import torch
from tqdm import tqdm
from sklearn.preprocessing import RobustScaler,StandardScaler,MinMaxScaler

In [ ]:
device='cuda' if torch.cuda.is_available() else 'CPU'

if device=='cuda':
  torch.cuda.manual_seed_all(0)

#Data Processing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
base_path = '/content/drive/MyDrive/Colab Notebooks/DeepLearning'

In [ ]:
# 파일 불러오기
def dataframe_from_csv(target):
  return pd.read_csv(target).rename(columns=lambda x:x.strip())

def dataframe_from_csvs(targets):
  return pd.concat([dataframe_from_csv(x) for x in targets])

train = pd.read_csv(base_path+'/source.csv',encoding='cp949')
print(f'train: {len(train)}')

train: 2060


In [ ]:
# train을 target과 feature로 나눠줍니다.
X=train.drop(['중증도'], axis=1).copy()
y=train['중증도']

In [ ]:
print("shape of x: {}\nshape of y: {}".format(X.shape,y.shape))

shape of x: (2060, 8)
shape of y: (2060,)


In [ ]:
from sklearn.model_selection import train_test_split
train_size=0.8
X_train, X_val, y_train, y_val = train_test_split(X,y, train_size=0.8)

In [ ]:
#feature scaling

#from sklearn.preprocessing import StandardScaler
#sc = StandardScaler()
#X_train = sc.fit_transform(X_train)
#X_val = sc.fit_transform(X_val)


In [ ]:
df_x_train=X_train.values
df_x_val=X_val.values

In [ ]:
torch_x_train = torch.FloatTensor(df_x_train).to(device)
torch_y_train = torch.LongTensor(y_train.to_numpy()).to(device)
torch_x_val = torch.FloatTensor(df_x_val).to(device)
torch_y_val = torch.LongTensor(y_val.to_numpy()).to(device)

In [ ]:
# 데이터 로더 지정
from torch.utils.data import DataLoader, TensorDataset

train_dataset = TensorDataset(torch_x_train, torch_y_train)
train_loader = DataLoader(train_dataset, batch_size=32)

test_dataset = TensorDataset(torch_x_val, torch_y_val)
test_loader = DataLoader(test_dataset, batch_size=32)

In [ ]:
# NN 모델 설계 : layer 생성
linear1 = torch.nn.Linear(8,1024)
linear2 = torch.nn.Linear(1024,1024)
linear3 = torch.nn.Linear(1024,512)
linear4 = torch.nn.Linear(512,512)
linear5 = torch.nn.Linear(512,128)
linear6 = torch.nn.Linear(128,2)
relu = torch.nn.ReLU()
dropout = torch.nn.Dropout(p = 0.18)

In [ ]:
# 가중치 초기화
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)
torch.nn.init.xavier_uniform_(linear4.weight)
torch.nn.init.xavier_uniform_(linear5.weight)
torch.nn.init.xavier_uniform_(linear6.weight)

Parameter containing:
tensor([[-0.0291,  0.1029, -0.1150,  0.0266,  0.0479,  0.0924,  0.1280, -0.1221,
         -0.1801, -0.1893,  0.0038,  0.1240,  0.1980,  0.1227,  0.1232, -0.0775,
         -0.1915,  0.2044,  0.1208,  0.0165,  0.1744, -0.0996, -0.0586,  0.1498,
          0.0274,  0.0693,  0.1591, -0.1103,  0.1636, -0.0416, -0.1545, -0.1826,
          0.1466,  0.0434,  0.0535,  0.0083,  0.0975, -0.1348,  0.0995, -0.1218,
          0.1025,  0.0754, -0.1005,  0.0528,  0.2038, -0.1067, -0.1613,  0.0894,
         -0.0160, -0.0567, -0.1981, -0.0825, -0.1854, -0.0837, -0.1508,  0.0627,
          0.1197, -0.1633, -0.1863,  0.1447,  0.0625,  0.1323, -0.0309, -0.1212,
          0.1050, -0.0515,  0.0774,  0.1285, -0.0459,  0.1368, -0.1032, -0.0192,
         -0.1014,  0.0764,  0.1645, -0.1097, -0.0748, -0.0674,  0.1371,  0.0172,
          0.1883, -0.1185,  0.0529, -0.0761,  0.1012,  0.1637, -0.0266, -0.0790,
         -0.0588, -0.1305,  0.0188,  0.1357,  0.0098,  0.0947, -0.0929,  0.0852,
      

In [ ]:
# 모델 생성
model = torch.nn.Sequential(linear1, relu, dropout,
                            linear2, relu, dropout,
                            linear3, relu, dropout,
                            linear4, relu, dropout,
                            linear5, relu, dropout,
                            linear6).to(device)

In [ ]:
# loss 및 optimzer 지정, learning rate & epoch 값 지정
loss = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.002)
epoch = 30

In [ ]:
# 학습
model.train()
total_batch = len(train_loader)

for i in range(epoch+1):
  avg_cost = 0

  for X, y in tqdm(train_loader):
    X = X.to(device)
    y = y.to(device)
    
    # H(x) 계산
    H = model(torch_x_train)
    print("====")
    print(H)
    print(H.shape)
    # cost 계산
    cost = loss(H, torch_y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch

  print("Epoch :", i, "Cost :", format(avg_cost), "Accuracy", )

 19%|█▉        | 10/52 [00:00<00:02, 16.71it/s]

====
tensor([[  45.2387,   77.2584],
        [ 374.3133,  171.9977],
        [-178.9211,  513.0334],
        ...,
        [ 343.4890, 1081.7874],
        [-666.2362,   -3.8852],
        [  -1.2233,    3.7670]], device='cuda:0', grad_fn=<AddmmBackward0>)
torch.Size([1648, 2])
====
tensor([[11770.7119, -8593.8721],
        [ 7841.8491, -5850.5801],
        [ 3505.1382, -2691.1943],
        ...,
        [ 9932.3506, -6522.2095],
        [ 8877.7383, -5497.1060],
        [   37.9092,   -38.2878]], device='cuda:0', grad_fn=<AddmmBackward0>)
torch.Size([1648, 2])
====
tensor([[ 5.9771e+02, -5.7936e+02],
        [ 2.3003e+03, -4.9057e+02],
        [ 8.1233e+02, -1.5372e+02],
        ...,
        [ 9.6208e+02, -1.7001e+02],
        [ 1.1902e+03, -9.2299e+02],
        [ 5.7824e+00, -9.7667e-01]], device='cuda:0', grad_fn=<AddmmBackward0>)
torch.Size([1648, 2])
====
tensor([[  90.7980, 1397.2361],
        [-431.8090, 1032.7979],
        [ 185.4553,  672.6296],
        ...,
        [ 356.9128, 11

 60%|█████▉    | 31/52 [00:00<00:00, 48.81it/s]

====
tensor([[170.6242, 153.2721],
        [ 98.1888,  27.1112],
        [ 31.5986,  31.5514],
        ...,
        [ 90.7365,  37.3100],
        [171.5489, 194.3494],
        [  0.6704,   0.7107]], device='cuda:0', grad_fn=<AddmmBackward0>)
torch.Size([1648, 2])
====
tensor([[-13.9305,  -4.8940],
        [133.6871, 110.1684],
        [ 55.3893,  77.7046],
        ...,
        [ 66.9174,  74.2255],
        [207.0556, 177.3785],
        [  0.7452,   0.6197]], device='cuda:0', grad_fn=<AddmmBackward0>)
torch.Size([1648, 2])
====
tensor([[130.8287, 107.1488],
        [ -8.9933, -28.1480],
        [ 56.7703,  69.3811],
        ...,
        [193.2433, 181.4695],
        [156.3043, 166.7227],
        [  0.5161,   0.9250]], device='cuda:0', grad_fn=<AddmmBackward0>)
torch.Size([1648, 2])
====
tensor([[ 1.3755e+02,  1.2670e+02],
        [ 3.4549e+01,  2.4983e+01],
        [ 5.2661e+00, -6.9764e+00],
        ...,
        [ 7.9409e+00, -1.2106e+01],
        [ 1.4527e+02,  1.4520e+02],
        [ 

100%|██████████| 52/52 [00:01<00:00, 44.06it/s]


...

====
tensor([[26.0203, 25.5026],
        [16.4312, 15.9861],
        [12.1996, 11.6352],
        ...,
        [22.7099, 22.1656],
        [11.6881, 11.0551],
        [ 0.6158,  0.0443]], device='cuda:0', grad_fn=<AddmmBackward0>)
torch.Size([1648, 2])
====
tensor([[ 2.2169,  1.6380],
        [15.6300, 15.1867],
        [12.4060, 11.8406],
        ...,
        [ 1.8357,  1.2268],
        [18.3737, 17.9109],
        [ 0.3724, -0.3516]], device='cuda:0', grad_fn=<AddmmBackward0>)
torch.Size([1648, 2])
====
tensor([[29.7115, 29.5176],
        [ 0.3693, -0.3552],
        [14.2194, 13.7276],
        ...,
        [ 0.3693, -0.3552],
        [ 0.3693, -0.3552],
        [ 0.3757, -0.3481]], device='cuda:0', grad_fn=<AddmmBackward0>)
torch.Size([1648, 2])
====
tensor([[36.2709, 36.0694],
        [19.4133, 18.9661],
        [ 0.9251,  0.2370],
        ...,
        [28.9084, 28.6833],
        [ 0.3694, -0.3553],
        [ 0.4135, -0.3103]], device='cuda:0', grad_fn=<AddmmBackward0>)
torch.Size([16

 87%|████████▋ | 45/52 [00:00<00:00, 106.71it/s]

tensor([[35.9525, 35.8624],
        [ 0.8136,  0.1512],
        [ 9.4450,  8.8743],
        ...,
        [ 2.3497,  1.9267],
        [18.5311, 18.1827],
        [ 0.4244, -0.3057]], device='cuda:0', grad_fn=<AddmmBackward0>)
torch.Size([1648, 2])
====
tensor([[27.1559, 26.7425],
        [13.2047, 12.5725],
        [13.0854, 12.4523],
        ...,
        [ 0.5393, -0.1696],
        [19.7097, 19.3174],
        [ 0.3749, -0.3565]], device='cuda:0', grad_fn=<AddmmBackward0>)
torch.Size([1648, 2])
====
tensor([[ 0.3731, -0.3590],
        [17.3904, 17.0016],
        [13.7073, 13.0721],
        ...,
        [15.9614, 15.5035],
        [ 2.0452,  1.5315],
        [ 0.3760, -0.3558]], device='cuda:0', grad_fn=<AddmmBackward0>)
torch.Size([1648, 2])


100%|██████████| 52/52 [00:00<00:00, 105.62it/s]

====
tensor([[20.5342, 20.1122],
        [ 0.3733, -0.3592],
        [16.8178, 16.3654],
        ...,
        [18.0811, 17.6194],
        [30.7754, 30.2829],
        [ 0.5029, -0.2512]], device='cuda:0', grad_fn=<AddmmBackward0>)
torch.Size([1648, 2])
====
tensor([[ 1.3466,  0.7603],
        [29.8031, 29.6421],
        [ 1.1078,  0.4856],
        ...,
        [27.7618, 27.3574],
        [26.8342, 26.4355],
        [ 0.4355, -0.2963]], device='cuda:0', grad_fn=<AddmmBackward0>)
torch.Size([1648, 2])
====
tensor([[38.5895, 38.4942],
        [23.2908, 23.0048],
        [ 3.8961,  3.3731],
        ...,
        [23.2945, 22.9630],
        [21.3866, 20.9816],
        [ 0.3752, -0.3580]], device='cuda:0', grad_fn=<AddmmBackward0>)
torch.Size([1648, 2])
====
tensor([[18.7376, 18.3641],
        [24.3287, 23.7915],
        [17.4044, 16.9279],
        ...,
        [ 0.3741, -0.3600],
        [13.5784, 13.2765],
        [ 0.3785, -0.3548]], device='cuda:0', grad_fn=<AddmmBackward0>)
torch.Size([16

In [ ]:
model.eval()

test_loss=0
correct=0

with torch.no_grad():
    for data,target in test_loader:
        data,target = data.to(device), target.to(device)
        output=model(data)
        test_loss+=F.nll_loss(output,target,reduction="sum").item()
        pred=output.argmax(dim=1,keepdim=True)
        correct+=pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)
print("\nTest set : Average Loss : {0}, Accuracy : {1}/{2}\n".format(test_loss,correct,len(test_loader.dataset)))



Test set : Average Loss : -34.87250807678815, Accuracy : 277/412

